In [1]:
"""[summary]
The script is to compute the cosine simliarities between queries and the document
The steps are as follows:
1. read the tf file
2. obtaint the input query
3. compute the cosine simliarities between query and each document
4.sort by the one with highest simliarity with the highest rank

"""

'[summary]\nThe script is to compute the cosine simliarities between queries and the document\nThe steps are as follows:\n1. read the tf file\n2. obtaint the input query\n3. compute the cosine simliarities between query and each document\n4.sort by the one with highest simliarity with the highest rank\n\n'

In [2]:
def readFile(_filePath):
    """[summary]

    Args:
        _filePath ([type]): [description]
    """
    import pandas as pd
    readFile = pd.read_csv(_filePath,index_col=0)
    return(readFile)

In [3]:
def queryVector(query,corpus):
    """[summary]
    vectorize query based on corpus
    
    Args:
        query ([list]): [list containing query terms]
    """
    corpusList = list(corpus.index)
    ## vector to track the corpusList
    vector = [0]*len(corpusList)
    ## format query\r\n",
    ## import scripts to process queries\r\n"
    import utils.query_processing
    ## convert to string if not string\r\n"
    if type(query) != str:
        query = ' '.join(query)
    ## clean the query
    glove_kv = '../pretrained_corpus/glove_6B_300d.kv'
    query = utils.query_processing.process_query(query)

    "still lacking the query expansion"
    "Need  glove_kv and topn to be saved in the repo"
    query = utils.query_processing.expand_query(query,glove_kv,3)    
    
    ## iterate over query terms
    for wordOfQuery in query:
    ## variable to tract the corpus index
        vectorIndex = 0
    ## iterate over the all corupus
        for wordOfCorpus in corpusList:
            if wordOfQuery ==  wordOfCorpus:
                vector[vectorIndex] += 1 
    ## update courpus index
            vectorIndex += 1               
    return(vector)

In [4]:
def cosineSimilarities(queryVector,docVector):
    "    \"\"\"[summary]\r\n"
    "    To compute the cosine similarity of the query with a single column/module in corpus\r\n"
    "    Args:\r\n"
    "        queryVector ([list]): [query]\r\n"
    "        docVector ([list]): [description]\r\n"
    from scipy import spatial
    distance = 1 - spatial.distance.cosine(queryVector, docVector)
    return(round(distance,5))

In [5]:
def topModulesForEachQuery(_df):
    "[summary]\r\n",
    "    To obtain the top modules for each of the query\r\n"
    "    Args:\r\n"
    "        _df ([dataframe]): [description] data with rows of words and columns contain the modules, the cell of the dataframe is the similarity\r\n"
    overallTopModules = {}
    for query in _df.columns:
        singleModule = {}
        df.sort_values(by = query,ascending = False, inplace = True)
        ## obtain top 10 modules
        topModules = df[query][:10]
        singleModule['topModules'] = list(topModules.index)
        singleModule['topModulesScore'] = list(topModules)
        overallTopModules[str(query)] = (singleModule)
        
    import pandas as pd
    overallTopModulesDf = pd.DataFrame(overallTopModules)
    return(overallTopModulesDf)

In [6]:
if __name__ == "__main__":
    import pandas as pd 
    corpus = readFile('../data/course_info_scores/course_info_tf.csv') ## <-- assign the merge corupus here
    # corpus = readFile('../data/course_info_with_survey_scores/course_info_with_survey_tf.csv') ## <-- assign the merge corupus here
    
    ## vaildation set
    """[summary]
    U can contorl which vaildation set to choose to test the cosine similarity model
    True : would be for the testingQuery that is self create which ensure a value when computing cosineSimilarities
    False : obtain the query based on 50% of vaildation dataset which might have all cosine similarities of zero
    """
    if False:
        testingQuery1 = ['ability','able','abstract']
        testingQuery2 = ['abstraction','accommodate','accompanies']
        testingQuery3 = ['account','accounting','achieve']
        testingQueryList = [[testingQuery1],[testingQuery2],[testingQuery3]]
        testingQueryDf = pd.DataFrame.from_records(testingQueryList,columns = ['querySample'])
        queries = testingQueryDf
        
    else:
        vaildationQueries = readFile('../data/survey/vaildation_sample_query.csv')
        queries = vaildationQueries
    
    """[summary]
    The start of the code where we can convert it to function
    """
    ## iterate over the vaildationQueries
    queriesScore = {}
    for row,query in queries.T.iteritems():
        print('\nCurrent computing Query: {}'.format(query['querySample']))
        singleQueryVector = queryVector(query['querySample'],corpus)
        print("Number of terms in corpus: {}".format(sum(singleQueryVector)))
        
        CosineSimilaritiesScore = []
        for modules in list(corpus):
            SimilaritiesScore = cosineSimilarities(singleQueryVector,corpus[modules])
            CosineSimilaritiesScore.append(SimilaritiesScore)
            # print("Score for Query on {} : {}".format(modules,SimilaritiesScore))
        queriesScore[str(query['querySample'])] = CosineSimilaritiesScore
        
    ## Convert to dict to df
    modules = list(corpus)
    df  = pd.DataFrame.from_dict(queriesScore,orient='index',columns=modules)
    df = df.T
    
    ##obtaining top modules for each query
    rankedModule = topModulesForEachQuery(df)
    rankedModule



Current computing Query: network, term, model, technology, probability


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Number of terms in corpus: 8

Current computing Query: term, different, skill, mongodb, long


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Number of terms in corpus: 7

Current computing Query: logistics, analysis, operation, basic, r


...

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Number of terms in corpus: 8

Current computing Query: program, future, kera, ec, concept


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Number of terms in corpus: 6


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [7]:
rankedModule

,"network, term, model, technology, probability","term, different, skill, mongodb, long","logistics, analysis, operation, basic, r","esd, network, evaluate, program, r","operation, basic, evaluation, price, evaluate","infrastructure, metric, pytorch, model, client","approach, logistics, shag, infrastructure, equity","data, sql, different, analytics, model","computational, best, server, certain, ec","sklearn, metric, demand, schedule, fundamental",...,"r, linear, ethereum, c, machine","science, future, problem, demand, pytorch","r, ec, kera, opponent, model","markov, descent, algebra, math, research","clojurescript, ai, analysis, background, science","optimisation, opencv, risk, chain, urban","maing, descent, science, clojurescript, knowledge","schedule, gradient, airport, analysis, system","logistics, business, strategy, analytics, math","program, future, kera, ec, concept"
topModules,"[50.012 Networks, 01.104 Networked Life, 40.31...","[40.240 Investment Science, 40.232 Water Resou...","[50.038 Computational Data Science, 50.043 Dat...","[50.012 Networks, 50.035 Computer Vision, 50.0...","[40.230 Sustainable Engineering, 01.102 Energy...","[40.319 Statistical and Machine Learning, 40.3...",[40.242 Derivative Pricing and Risk Management...,"[50.038 Computational Data Science, 50.043 Dat...","[50.048 Computational Fabrication, 01.117 Brai...","[40.320 Airport Systems Planning and Design, 4...",...,"[50.007 Machine Learning, 40.319 Statistical a...","[50.021 Artificial Intelligence, 40.302 Advanc...","[40.319 Statistical and Machine Learning, 40.3...",[40.305 Advanced Topics in Stochastic Modellin...,"[50.038 Computational Data Science, 01.116 AI ...","[40.302 Advanced Topics in Optimisation#, 40.2...","[50.021 Artificial Intelligence, 50.038 Comput...","[50.038 Computational Data Science, 40.321 Air...",[40.318 Supply Chain Digitalisation and Design...,"[50.041 Distributed Systems and Computing, 40...."
topModulesScore,"[0.33558, 0.27537, 0.26261, 0.25574, 0.25503, ...","[0.10359, 0.0927, 0.08893, 0.08321, 0.06656, 0...","[0.31274, 0.18262, 0.15149, 0.14639, 0.12689, ...","[0.32287, 0.19019, 0.18865, 0.15822, 0.12388, ...","[0.1575, 0.1297, 0.11734, 0.09082, 0.08819, 0....","[0.22743, 0.22086, 0.20025, 0.19181, 0.18464, ...","[0.15525, 0.13622, 0.12481, 0.08607, 0.04568, ...","[0.3872, 0.37829, 0.23451, 0.23212, 0.22257, 0...","[0.18794, 0.11128, 0.10076, 0.06491, 0.06131, ...","[0.18443, 0.14062, 0.08802, 0.08588, 0.08584, ...",...,"[0.18765, 0.11606, 0.11169, 0.10401, 0.04541, ...","[0.24776, 0.20089, 0.15986, 0.12317, 0.1182, 0...","[0.32163, 0.31235, 0.2832, 0.27126, 0.257, 0.2...","[0.1704, 0.10044, 0.04255, 0.03552, 0.03197, 0...","[0.33772, 0.31972, 0.17749, 0.12491, 0.10515, ...","[0.22549, 0.17282, 0.15646, 0.13007, 0.06587, ...","[0.12746, 0.08935, 0.08115, 0.07538, 0.06964, ...","[0.25329, 0.23328, 0.19228, 0.16599, 0.12053, ...","[0.19256, 0.1482, 0.06376, 0.05344, 0.03211, 0...","[0.12372, 0.11043, 0.10242, 0.07204, 0.06726, ..."
